# 01 – Limpieza y EDA (Python)

Este notebook carga los CSVs, limpia datos (nulos, espacios, mayúsculas/minúsculas, duplicados),
crea una tabla de **hechos** de ventas y dimensiones (clientes y productos), y exporta a `data/processed/` para SQL/Power BI.


In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path

BASE = Path().resolve()
DATA_RAW = BASE / 'data' / 'raw'
DATA_PROC = BASE / 'data' / 'processed'
DATA_PROC.mkdir(parents=True, exist_ok=True)

customers = pd.read_csv(DATA_RAW/'customers.csv')
products = pd.read_csv(DATA_RAW/'products.csv')
orders = pd.read_csv(DATA_RAW/'orders.csv', parse_dates=['order_datetime'])
order_lines = pd.read_csv(DATA_RAW/'order_lines.csv')


In [ ]:

# --- Limpieza básica ---
# Trim espacios
for col in ['payment_method']:
    orders[col] = orders[col].astype(str).str.strip()

# Normalizar canal
orders['channel'] = orders['channel'].astype(str).str.strip().str.title()

# Rellenar ciudades faltantes con 'Desconocido'
customers['city'] = customers['city'].fillna('Desconocido')

# Quitar duplicados de órdenes
before = len(orders)
orders = orders.drop_duplicates(subset=['order_id','customer_id','order_datetime','channel','payment_method','status'])
after = len(orders)
print(f'Removed {before-after} potential duplicated rows in orders')

# --- Derivados de fecha ---
orders['order_date'] = orders['order_datetime'].dt.date
orders['order_month'] = orders['order_datetime'].dt.to_period('M').astype(str)
orders['order_year'] = orders['order_datetime'].dt.year

# --- Construir tabla de hechos uniendo con order_lines ---
fact = order_lines.merge(orders[['order_id','customer_id','order_datetime','order_date','order_month','order_year','channel','payment_method','status']], on='order_id', how='inner')
fact = fact.merge(products[['product_id','product_name','category']], on='product_id', how='left')
fact = fact.merge(customers[['customer_id','city','segment','signup_date']], on='customer_id', how='left')

# Filtrar solo ventas completadas para métricas principales
fact_completed = fact.query("status == 'Completed'").copy()

# Exportar procesados
customers.to_csv(DATA_PROC/'dim_customers.csv', index=False)
products.to_csv(DATA_PROC/'dim_products.csv', index=False)
orders.to_csv(DATA_PROC/'stg_orders_clean.csv', index=False)
fact.to_csv(DATA_PROC/'fact_sales_all_status.csv', index=False)
fact_completed.to_csv(DATA_PROC/'fact_sales.csv', index=False)

fact_completed.head()


In [ ]:

# Chequeos rápidos de calidad
print('Nulls in fact_sales:')
print(fact_completed.isna().sum().sort_values(ascending=False).head(10))

print('\nRangos de fechas: ', fact_completed['order_datetime'].min(), ' -> ', fact_completed['order_datetime'].max())

# KPIs simples
kpis = {
    'orders': fact_completed['order_id'].nunique(),
    'customers': fact_completed['customer_id'].nunique(),
    'revenue': round(fact_completed['line_revenue'].sum(),2),
    'profit': round(fact_completed['line_profit'].sum(),2)
}
kpis
